In [ ]:
import pandas as pd

## Data Cleaning

### 1. OECD Data

Logic to extract data of pre-selected countries and indicators from raw OECD (The Organisation for Economic Co-operation and Development) raw data

In [ ]:
def extract_analysis_data_from_df(df, year):
    location = {
        'AUS':'Australia', 
        'USA':'United States',
        'CAN':'Canada',
        'DEU':'Germany',
        'CHE':'Switzerland',
        'FRA':'France',
        'ESP':'Spain',
        'ITA':'Italy',
        'SWE':'Sweden',
        'GBR':'United Kingdom'
    }
    indicators = {
        'HO_HISH':'Housing expenditure',
        'HO_NUMR':'Rooms per person',
        'JE_EMPL':'Employment rate',
        'JE_LTUR':'Long-term unemployment rate',
        'EQ_AIRP':'Air pollution',
        'EQ_WATER':'Water quality',
        'HS_LEB':'Life expectancy',
        'SW_LIFS':'Life satisfaction',
        'WL_EWLH':'Employees working very long hours',
        'WL_TNOW':'Time devoted to leisure and personal care',
        'PS_FSAFEN':'Feeling safe walking alone at night',
        'HS_SFRH':'Self-reported health'
    }
    df_clean = df.loc[df['LOCATION'].isin(location.keys())]\
                .loc[df['INDICATOR'].isin(indicators.keys())]\
                .loc[df['Inequality']=='Total'][['Country','Indicator','Value']]\
                .sort_values(['Country','Indicator'])
    df_clean['year'] = year
    return df_clean

Reading raw OECD data files from 2013 to 2016 into pandas dataframe

In [ ]:
df2016 = pd.read_csv("./source_files/BLI_2016.csv")
df2015 = pd.read_csv("./source_files/BLI_2015.csv")
df2014 = pd.read_csv("./source_files/BLI_2014.csv")
df2013 = pd.read_csv("./source_files/BLI_2013.csv")

Extracting required records from raw data files

In [ ]:
clean_df2016 = extract_analysis_data_from_df(df2016, 2016)
clean_df2015 = extract_analysis_data_from_df(df2015, 2015)
clean_df2014 = extract_analysis_data_from_df(df2014, 2014)
clean_df2013 = extract_analysis_data_from_df(df2013, 2013)

Merge cleaned data for all years into a dataframe and save to CSV

In [ ]:
frames = [clean_df2016, clean_df2015, clean_df2014, clean_df2013]
merged_df = pd.concat(frames)
merged_df.to_csv("./processed_files/oced_clean_data.csv",index = False)

### 2. Immigration Data

Read raw immigration data retrieved from OECD, extract required indicators for pre-selected countries

In [ ]:
df_immigration = pd.read_csv("./source_files/OCED_IMMIGRATION.csv", low_memory=False)
variables = ['Inflows of foreign population by nationality',
             'Acquisition of nationality by country of former nationality']
location = {
        'AUS':'Australia', 
        'USA':'United States',
        'CAN':'Canada',
        'DEU':'Germany',
        'CHE':'Switzerland',
        'FRA':'France',
        'ESP':'Spain',
        'ITA':'Italy',
        'SWE':'Sweden',
        'GBR':'United Kingdom'
    }

# For Immigration we are targeting people moved from India to one of the above mentioned countries
df_filtered = df_immigration.loc[df_immigration['Country of birth/nationality']=='India']\
.loc[df_immigration['Country'].isin(location.values())]\
.loc[df_immigration['Variable'].isin(variables)][['Variable','Country','Year','Value']]

Reformatting immigration data as it has two values for each country - 1. Inflow, 2. Nationality

In [ ]:
df_immi_inflow = df_filtered.loc[df_immigration['Variable'] == 'Inflows of foreign population by nationality'][['Country','Year','Value']]
df_immi_acq_nationality = df_filtered.loc[df_immigration['Variable'] == 'Acquisition of nationality by country of former nationality'][['Country','Year','Value']]

df_immi_inflow.rename(columns={'Value':'Inflow'}, inplace=True)
df_immi_acq_nationality.rename(columns={'Value': 'Nationality'}, inplace=True)

df_merged = pd.merge(df_immi_inflow, df_immi_acq_nationality, on=['Country','Year'], how='left')

Calculate % of people recieved nationality based on Inflow and Nationality

In [ ]:
df_merged['Perc_In_To_Nationality'] = df_merged['Nationality'] * 100 /df_merged['Inflow']
#Replace NaN with 0
df_merged = df_merged.fillna(0)

Write cleaned immigration data to CSV

In [ ]:
df_merged.to_csv("./processed_files/immigration_data.csv",index = False)

### 3. Global Innovation Index

Read raw data files into dataframe

In [ ]:
location['USA'] = 'United States of America'
df_biz = pd.read_csv("./source_files/EASE_START_BIZ_2016.csv")
df_gii = pd.read_csv("./source_files/GLOBAL_INNOVATION_INDEX_2016.csv")
df_no_terror = pd.read_csv("./source_files/POLITICAL_STA_NO_TERROR_2016.csv")
df_rnd = pd.read_csv("./source_files/R&D_SPEND_2016.csv")

Filter records for pre-selected countries for all raw dataframes and rename columns to standard schema

In [ ]:
df_biz = df_biz.loc[df_biz['Economy'].isin(location.values())][['Economy','Score']]
df_biz.rename(columns={'Economy':'Country','Score':'Value'}, inplace=True)
df_biz['Indicator'] = "Ease of starting a new business"

In [ ]:
df_gii = df_gii.loc[df_gii['Economy'].isin(location.values())][['Economy','Score']]
df_gii.rename(columns={'Economy':'Country','Score':'Value'}, inplace=True)
df_gii['Indicator'] = "Global Innovation Index"

In [ ]:
df_no_terror = df_no_terror.loc[df_no_terror['Economy'].isin(location.values())][['Economy','Score']]
df_no_terror.rename(columns={'Economy':'Country','Score':'Value'}, inplace=True)
df_no_terror['Indicator'] = "Political Stability/Absence of Terrorism"

In [ ]:
df_rnd = df_rnd.loc[df_rnd['Economy'].isin(location.values())][['Economy','Value']]
df_rnd.rename(columns={'Economy':'Country'}, inplace=True)
df_rnd['Indicator'] = "R&D Spend - % of GDP"

Merge cleaned data into single dataframe and write to CSV

In [ ]:
df_gobal_innovation_index = pd.concat([df_biz,df_gii,df_no_terror,df_rnd])

In [ ]:
df_gobal_innovation_index.loc[df_gobal_innovation_index['Country'] == 'United States of America', 'Country'] = 'United States'
df_gobal_innovation_index.to_csv("./processed_files/global_innovation_index.csv",index = False)

### 4. Salary

Read raw data, add calculated filed and write back to CSV

In [ ]:
df_salary = pd.read_csv("./source_files/SE_SALARY.csv")
df_salary['PERC_MORE_THAN_AVG'] = df_salary['MEDIAN_PAY_SE']*100/df_salary['AVG_INCOME']
df_salary.to_csv("./processed_files/salary_se.csv",index = False)

## Data Transformation/Scaling

In [ ]:
from sklearn import preprocessing

Scaling is done based on relative distance between values of each indicator for all Countries. I am using scikit-learn `MinMaxScaler` to scale the valeus between 10 and 100. 

For one indicator, for one year, the country with minimum value will get 10 and the country with maximum value will get 100 as scaled value. Scaled value of all other countries will be assigned based on their relative distance from Min, Max and each other. Please refer http://scikit-learn.org/stable/modules/preprocessing.html#scaling-features-to-a-range for more information.

In [ ]:
def scale_df(df):
    x = df.values #returns a numpy array
    # scikit-learn MinMaxScaler is used to scale the valeus between 10 and 100.
    # Min value will get 10 and max will get 100. 
    # All other values will be assigned based on relative distance
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0.1, 1))
    x = x.reshape(-1,1)
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled * 100

#### Scaling OECD Data

In [ ]:
df = pd.read_csv("./processed_files/oced_clean_data.csv")

While comparing countries, an indicator can either be a positive or negative factor. To maintain consistancy in comparision and allow cross comparision between any two indicators, I identified factors which are negative and applied reverse scaling to convert them into positive factors. The code below scales values of each indicator based on either it is a positive or negative.

In [ ]:
# indicators => key = indicator, value = is key a positive indicator?
indicators = {
    'Housing expenditure': False,
    'Employment rate': True,
    'Long-term unemployment rate': False,
    'Air pollution': False,
    'Water quality': True,
    'Life expectancy': True,
    'Life satisfaction': True,
    'Time devoted to leisure and personal care':True,
    'Feeling safe walking alone at night':True,
    'Self-reported health': True
}

oced_years = [2013, 2014, 2015, 2016]
df_dict_indicator = {}
df_with_scaled_value = []
for indicator, is_positive in indicators.iteritems():
    for year in oced_years:
        tmp_df = df.loc[df['Indicator'] == indicator].loc[df['year'] == year]
        if not tmp_df.empty:
            if not is_positive:
                original_values = tmp_df['Value'].copy(deep=True)
                tmp_df['Value'] = tmp_df['Value'].apply(lambda x: 100-x)
            tmp_df['ScaledValue'] = scale_df(tmp_df['Value'])
            tmp_df['ScaledValue'] = tmp_df['ScaledValue'].round(2)
            if not is_positive:
                tmp_df['Value'] = original_values
            df_with_scaled_value.append(tmp_df)

df_final = pd.concat(df_with_scaled_value)
df_final.to_csv("./processed_files/scaled_oced_clean_data.csv", index=False)

#### Scaling Immigration Data

Similar logic is also used to scale value of immigration data

In [ ]:
df_immi = pd.read_csv("./processed_files/immigration_data.csv")

In [ ]:
df_with_rank = []
immi_years = df_immi.Year.unique()

for year in immi_years:
    tmp_df = df_immi.loc[df_immi['Year'] == year]
    if not tmp_df.empty:
        tmp_df['ScaledValueInflow'] = scale_df(tmp_df['Inflow'])
        tmp_df['ScaledValueInflow'] = tmp_df['ScaledValueInflow'].round(2)
        tmp_df['ScaledValueNationality'] = scale_df(tmp_df['Nationality'])
        tmp_df['ScaledValueNationality'] = tmp_df['ScaledValueNationality'].round(2)
        df_with_rank.append(tmp_df)

df_immi_final = pd.concat(df_with_rank)
df_immi_final.to_csv("./processed_files/scaled_immigration_data.csv",index = False)

#### Scaling Salary Data

In [ ]:
df_salary = pd.read_csv("./processed_files/salary_se.csv")

In [ ]:
df_salary['ScaledMedianPay'] = scale_df(df_salary['MEDIAN_PAY_SE'])
df_salary['ScaledMedianPay'] = df_salary['ScaledMedianPay'].round(2)
df_salary.to_csv("./processed_files/scaled_salary_data.csv",index = False)

#### Scaling Global Innovation Index Data

In [ ]:
df_gii = pd.read_csv("./processed_files/global_innovation_index.csv")

In [ ]:
gii_indicators = df_gii.Indicator.unique()
df_with_rank = []
for indicator in gii_indicators:
    tmp_df = df_gii.loc[df_gii['Indicator'] == indicator]
    if not tmp_df.empty:
        tmp_df['ScaledValue'] = scale_df(tmp_df['Value'])
        tmp_df['ScaledValue'] = tmp_df['ScaledValue'].round(2)
        df_with_rank.append(tmp_df)

df_gii_final = pd.concat(df_with_rank)      
df_gii_final.to_csv("./processed_files/scaled_global_innovation_index_data.csv",index = False)

## Data Merge 

The final merged dataframe is generated by combining cleaned data from all four sources and taking the latest values of each factor for each country,

In [ ]:
df_oced = pd.read_csv("./processed_files/scaled_oced_clean_data.csv")
df_salary = pd.read_csv("./processed_files/scaled_salary_data.csv")
df_immigration = pd.read_csv("./processed_files/scaled_immigration_data.csv")
df_gii = pd.read_csv("./processed_files/scaled_global_innovation_index_data.csv")

In [ ]:
df_oced = df_oced.loc[df_oced['year'] == df_oced.year.max()][['Country','Indicator','Value', 'ScaledValue']]

In [ ]:
df_salary['Indicator'] = 'Median Pay'
df_salary = df_salary.rename(columns = {'COUNTRY':'Country', 'ScaledMedianPay': 'ScaledValue', 'MEDIAN_PAY_SE': 'Value'})[['Country','Indicator', 'Value', 'ScaledValue']]

In [ ]:
df_inflow = df_immigration.loc[df_immigration['Year'] == df_immigration.Year.max()-1][['Country','ScaledValueInflow','Inflow']]
df_inflow['Indicator'] = 'Immigration Inflow'
df_inflow = df_inflow.rename(columns = {'ScaledValueInflow': 'ScaledValue', 'Inflow': 'Value'})[['Country','Indicator', 'Value', 'ScaledValue']]

In [ ]:
df_gii = df_gii[['Country', 'Indicator', 'Value','ScaledValue']]

In [ ]:
df_list = [df_oced, df_salary, df_inflow, df_gii]
ranked_df = pd.concat(df_list)

## Data Categorization

Before write the merged dataframe to CSV file, a category is added for each factor based on categories mentioned in below table

Indicator | Category
--- | --- 
Self-reported health | Health
R&D Spend - % of GDP | Business & Innovation
Employment rate | Employment
Air pollution | Environment Health
Life expectancy | Health
Ease of starting a new business | Business & Innovation
Median Pay | Financial Health
Feeling safe walking alone at night | Safety
Immigration Inflow | Ease of Immigration
Global Innovation Index | Business & Innovation
Immigration Nationality | Ease of Immigration
Political Stability/Absence of Terrorism | Safety
Water quality | Environment Health
Housing expenditure | Financial Health
Time devoted to leisure and personal care | Happiness
Long-term unemployment rate | Employment
Life satisfaction | Happiness

In [ ]:
indicator_category = {
    'Employment rate': 'Employment',
    'Long-term unemployment rate': 'Employment',
    'Water quality': 'Environment Health',
    'Air pollution': 'Environment Health',
    'Life expectancy': 'Health',
    'Self-reported health': 'Health',
    'Political Stability/Absence of Terrorism':'Safety',
    'Feeling safe walking alone at night':'Safety',
    'Ease of starting a new business': 'Business & Innovation',
    'Global Innovation Index':'Business & Innovation',
    'R&D Spend - % of GDP':'Business & Innovation',
    'Housing expenditure':'Financial Health',
    'Median Pay':'Financial Health',
    'Life satisfaction':'Happiness',
    'Time devoted to leisure and personal care':'Happiness',
    'Immigration Inflow':'Immigration Inflow'
}

After assigning categories to each row, final dataframe is written to a CSV file

In [ ]:
ranked_df['Category'] = ""
for indicator, category in indicator_category.iteritems():
    ranked_df['Category'][ranked_df['Indicator'] == indicator]=category
    
ranked_df.to_csv("./processed_files/country_indicator_scaled_value.csv",index = False)

Pivoting the data frame to reformat data to find countries based on catagory's value in Tableau using slider

In [ ]:
input_df = pd.read_csv("./processed_files/country_indicator_scaled_value.csv")
categorized_df = input_df[['Country', 'Category', 'ScaledValue']].groupby(['Country', 'Category'], as_index=False).mean()
categorized_df['ScaledValue'] = categorized_df['ScaledValue'].round(2)
categorized_df = categorized_df.pivot_table('ScaledValue', ['Country'], 'Category')
categorized_df.reset_index( drop=False, inplace=True )
categorized_df = categorized_df.reindex_axis(['Country', 'Business & Innovation', 'Employment', 'Environment Health', 'Financial Health','Happiness','Health','Immigration Inflow', 'Safety'], axis=1)
categorized_df.to_csv("./processed_files/country_categorized_values.csv",index = False)